In [1]:
import pandas as pd

features_df = pd.read_csv("/home/j.maragall/PRESENTATION_FEATURES.csv")

import pandas as pd
from sklearn.preprocessing import MinMaxScaler


# Impute all NaN values with 0
features_df.fillna(0, inplace=True)

In [9]:
import numpy as np
import tifffile

# Assuming 'features_df' is your DataFrame and it has been properly defined

# Specify your mask dimensions (image dimensions)
mask_height = 17448  # Example height
mask_width = 4908    # Example width

# Create an empty binary mask
binary_mask = np.zeros((mask_height, mask_width), dtype=np.uint8)

# Fill in the regions specified in the DataFrame, ensuring indices are integers
for index, row in features_df.iterrows():
    # Convert slice indices to integers
    minr = int(row['minr'])
    maxr = int(row['maxr'])
    minc = int(row['minc'])
    maxc = int(row['maxc'])
    
    # Use these integer values to slice and update the binary mask
    binary_mask[minr:maxr, minc:maxc] = 255

# Specify the path and file name for the output TIFF file
output_file_path = 'features_binary_mask.tif'

# Save the binary mask as a TIFF file
tifffile.imwrite(output_file_path, binary_mask)

In [10]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tifffile import imread
from PIL import Image
from skimage.measure import regionprops, label
from skimage.feature import graycomatrix, graycoprops
from scipy.stats import entropy
from umap import UMAP
import tiffslide as openslide
from concurrent.futures import ProcessPoolExecutor

def normalize_image(image, target_dtype=np.uint8):
    normalized_image = cv2.normalize(image, None, 0, np.iinfo(target_dtype).max, cv2.NORM_MINMAX)
    return normalized_image.astype(target_dtype)

def load_and_process_images(image1_path, image2_path):
    slide1 = openslide.OpenSlide(image1_path)
    slide2 = openslide.OpenSlide(image2_path)
    img1 = normalize_image(np.array(slide1.get_thumbnail(slide1.dimensions)), np.uint8)
    img2 = normalize_image(np.array(slide2.get_thumbnail(slide2.dimensions)), np.uint8)
    return img1, img2

def compute_homography(img1, img2):
    sift = cv2.SIFT_create()
    kp1, d1 = sift.detectAndCompute(img1, None)
    kp2, d2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    matcher = cv2.FlannBasedMatcher(index_params, search_params)
    matches = matcher.knnMatch(d1, d2, k=2)

    good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]

    p1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    p2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    
    homography, _ = cv2.findHomography(p1, p2, cv2.RANSAC)
    return homography

def transform_patch(patch, channel_index, homography, patch_size):
    single_channel_img = patch[:,:,channel_index]
    normalized_single_channel_img = normalize_image(single_channel_img, np.uint8)
    transformed_single_channel_img = cv2.warpPerspective(normalized_single_channel_img, homography, (patch_size, patch_size))
    return transformed_single_channel_img, channel_index

def transform_multichannel_image(multi_channel_img, homography, patch_size=512):
    num_patches_width = multi_channel_img.shape[1] // patch_size
    num_patches_height = multi_channel_img.shape[0] // patch_size
    transformed_channels = [np.zeros_like(multi_channel_img[:, :, 0], dtype=np.uint8) for _ in range(multi_channel_img.shape[2])]

    with ProcessPoolExecutor(max_workers=20) as executor:
        futures = []
        for i in range(num_patches_height):
            for j in range(num_patches_width):
                y_min = i * patch_size
                y_max = (i + 1) * patch_size
                x_min = j * patch_size
                x_max = (j + 1) * patch_size

                patch = multi_channel_img[y_min:y_max, x_min:x_max, :]

                for channel_index in range(patch.shape[2]):
                    futures.append(executor.submit(transform_patch, patch, channel_index, homography, patch_size))
                    
        for future in concurrent.futures.as_completed(futures):
            transformed_single_channel_img, channel_index = future.result()
            y_min = i * patch_size
            y_max = (i + 1) * patch_size
            x_min = j * patch_size
            x_max = (j + 1) * patch_size
            transformed_channels[channel_index][y_min:y_max, x_min:x_max] = transformed_single_channel_img
    
    return np.stack(transformed_channels, axis=-1)
    return np.stack(transformed_channels, axis=-1)

def extract_features(binary_mask_path, registered_multi_channel_img):
    image = Image.open(binary_mask_path)
    image_array = np.array(image)
    labeled_image, number_of_objects = label(image_array == 255, connectivity=2, return_num=True)
    
    features_list = []

    for region_label in range(1, number_of_objects + 1):
        object_mask = (labeled_image == region_label)
        properties = regionprops(object_mask.astype(int))[0]
        area = properties.area
        perimeter = properties.perimeter
        eccentricity = properties.eccentricity
        solidity = properties.solidity
        orientation = properties.orientation

        channel_features = []

        for channel_index in range(registered_multi_channel_img.shape[2]):
            single_channel_img = registered_multi_channel_img[:, :, channel_index]
            masked_single_channel = single_channel_img * object_mask

            mean_intensity = np.mean(masked_single_channel[object_mask])
            std_intensity = np.std(masked_single_channel[object_mask])
            variance_intensity = np.var(masked_single_channel[object_mask])
            hist, _ = np.histogram(masked_single_channel[object_mask], bins=256, range=(0, 256), density=True)
            entropy_intensity = entropy(hist)
            glcm = graycomatrix(masked_single_channel, [1], [0], symmetric=True, normed=True)
            contrast = graycoprops(glcm, 'contrast')[0, 0]

            channel_features.extend([mean_intensity, std_intensity, variance_intensity, entropy_intensity, contrast])

        features_list.append(channel_features + [area, perimeter, eccentricity, solidity, orientation])

    return pd.DataFrame(features_list)

def visualize_umap_embedding(features_df):
    umap = UMAP(n_neighbors=5, min_dist=0.3, n_components=2, metric='euclidean')
    embedding = umap.fit_transform(features_df.values)

    plt.scatter(embedding[:, 0], embedding[:, 1], s=5)
    plt.title('UMAP Embedding of Extracted Features')
    plt.xlabel('UMAP Dimension 1')
    plt.ylabel('UMAP Dimension 2')
    plt.show()

if __name__ == "__main__":
    # Paths
    image1_path = "/home/j.maragall/Pipeline_CellMapping/features_binary_mask.tif"
    image2_path = "/home/j.maragall/Pipeline_CellMapping/R2_DAPI_cleaned_mask.tif"
    multi_channel_image_path = "/blue/pinaki.sarder/j.maragall/Replication1_DAPImapping/15-1.tif"
    binary_mask_path = image2_path
    
    # Load and process images
    img1, img2 = load_and_process_images(image1_path, image2_path)
    
    # Compute homography
    homography = compute_homography(img1, img2)
    
    # Transform multi-channel image
    multi_channel_img = imread(multi_channel_image_path)
    registered_multi_channel_img = transform_multichannel_image(multi_channel_img, homography)
    
    # Extract features
    features_df = extract_features(binary_mask_path, registered_multi_channel_img)
    
    # Visualize UMAP embedding
    visualize_umap_embedding(features_df)


/apps/jupyter/6.5.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-04 20:48:11.782995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 20:48:16.165389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

KeyboardInterrupt



In [7]:
pwd

'/home/j.maragall/Pipeline_CellMapping'